In [29]:
from itertools import product
from collections import Counter
import random
import requests
import json
import pandas as pd
import time
from tqdm import tqdm
# from tqdm.auto import tqdm
from getpass import getpass
# from google.colab import files

In [ ]:
scenario_data = json.load(open("eng_scenarios_50k.json"))
NUMBER_OF_SCENARIOS = 5000 # len(scenario_data)

## LMStudio requests

In [36]:
url = 'http://localhost:1234/v1/models'

headers = {
  'Accept': 'application/json',
}

response = requests.request("GET", url, headers=headers)

print(response.json())



{'data': [{'id': 'Orenguteng/Llama-3-8B-Lexi-Uncensored-GGUF/Lexi-Llama-3-8B-Uncensored_Q5_K_M.gguf', 'object': 'model', 'owned_by': 'organization-owner', 'permission': [{}]}], 'object': 'list'}


In [37]:
MODEL = "Orenguteng/Llama-3-8B-Lexi-Uncensored-GGUF/Lexi-Llama-3-8B-Uncensored_Q5_K_M.gguf"

In [39]:
def send_lmstudio_request(system_content, user_content):
  prompt = {
    "model": MODEL,
    "messages": [
      {
        "role": "system",
        "content": system_content
      },
      {
        "role": "user",
        "content": user_content
      }
    ],
    'stream': False,
    'max_tokens': 10,
      "temperature": 0.2
  }

  url = 'http://localhost:1234/v1/chat/completions'
  headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
  }

  response = requests.request("POST", url, headers=headers, data=json.dumps(prompt))
  # print(response.json())
  return response.json()["choices"][0]["message"]["content"]

In [40]:
scenario_list = []

In [ ]:
for i in tqdm(range(NUMBER_OF_SCENARIOS)):
  system_content = scenario_data[i]["system_content"]
  user_content = scenario_data[i]["user_content"]
  scenario_info = scenario_data[i]["scenario_info"]
    
  try:
    response = send_lmstudio_request(system_content, user_content)

    scenario_info["response"] = response
    scenario_list.append(scenario_info)
  except:
    print("Problem!")
    continue
  # time.sleep(0.2)

df = pd.DataFrame(scenario_list)
last_successfull_name = f"eng_response_{MODEL.replace('/', '')}_{len(scenario_list)}.pickle"
df.to_pickle(last_successfull_name)
# files.download(last_successfull_name)

  6%|▌         | 2984/50000 [34:03<9:46:45,  1.34it/s] 

In [ ]:
len(scenario_list)